<a target="_blank" href="https://colab.research.google.com/github/Andrew-mcArty/RL_AAPGAB/blob/main/estudio_montecarlo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!curl -O https://raw.githubusercontent.com/Andrew-mcArty/RL_AAPGAB/refs/heads/main/src/FrozenAgent.py

In [2]:
from __future__ import annotations
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
import gymnasium as gym
import FrozenAgent
import random
import time

semilla=100

ModuleNotFoundError: No module named 'FrozenAgent'

Cargamos el entorno

In [ ]:


print("definimos el entorno:")
name = 'FrozenLake-v1'
env4 = gym.make(name, is_slippery=False, map_name="4x4", render_mode="ansi") # No resbaladizo para entender mejor los resultados.
env8 = gym.make(name, is_slippery=False, map_name="8x8", render_mode="ansi") # No resbaladizo para entender mejor los resultados.


definimos el entorno:


Función útil para establecer la semilla tanto en numpy como en la librería random

In [ ]:

def setSemilla(semilla):
    random.seed(semilla)
    np.random.seed(semilla)


Algorimto genérico de para evaluar un agente

In [ ]:

def train_agent(agent, env, num_episodes=5000, decay=False, semilla=1):
    agent.initAgent()
    for episode in tqdm(range(num_episodes)):
        state, info = env.reset(seed=semilla)
        done = False
    
        start_time=time.time_ns()
        #inicializo el episodio
        agent.initEpisode()
    
        # play one episode
        while not done:
            if decay:
                agent.decay_epsilon()
            action = agent.get_action(env, state)
            
            next_state, reward, terminated, truncated, info = env.step(action)

            # update the agent
            agent.updateStep(state, action, reward, terminated, next_state)
            
            # update if the environment is done and the current state
            done = terminated or truncated
            state = next_state
            
        #después de acabar el episodio actualizo la Q y el epsilon
        agent.updateEpisode()


Muestra los resultados recogidos por el agente durante su entrenamiento. Esta función muestra dos indicadores que implementan todos los agentes: 
* proporción de recompensas: suma recompensas / episodios
* proporción de tamaño de episodios: suma len(episodio) / episodios

In [ ]:


def plot(agent):
  # Creamos una lista de índices para el eje x
  indices = list(range(len(agent.list_stats)))
  
  
  # Crear figura con dos subgráficos
  fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))

  # Primer subplot
  ax1.plot(indices, agent.list_stats, label='stats')
  ax1.set_title('Proporción de recompensas')
  ax1.set_xlabel('Episodio')
  ax1.set_ylabel('Proporción')
  ax1.legend()
  ax1.grid(True)

  # Segundo subplot
  ax2.plot(indices, agent.list_episodes, label='episodios')
  ax2.set_title('Tamaño de episodios')
  ax2.set_xlabel('Episodio')
  ax2.set_ylabel('Tamaño')
  ax2.legend()
  ax2.grid(True)

  # Ajustar diseño y mostrar gráfico
  plt.tight_layout()
  plt.show()  



Definimos dos agentes: MC on policy all visits y first visit.

Ejecutamos en entrenameiento y obtenemos la mejor política de cada uno

In [ ]:


#inicializo los numeros aleatorios
setSemilla(semilla)

# hyperparameters
n_episodes = 5000
start_epsilon = 0.1
discount_factor = 0.99

agent1 = FrozenAgent.FrozenAgentMC_On_All(
    env=env4,
    epsilon=start_epsilon,
    discount_factor=discount_factor,
)
agent2 = FrozenAgent.FrozenAgentMC_On_First(
    env=env4,
    epsilon=start_epsilon,
    discount_factor=discount_factor,
)

train_agent(agent1, env4, num_episodes=n_episodes, decay=False, semilla=semilla)
train_agent(agent2, env4, num_episodes=n_episodes, decay=False, semilla=semilla)

pi1, actions1 = agent1.pi_star_from_Q(env4, agent1.Q)
pi2, actions2 = agent2.pi_star_from_Q(env4, agent2.Q)


100%|██████████| 5000/5000 [00:02<00:00, 1945.39it/s]


Mostramos resultados

In [ ]:
print(f"================== AGENTE 1 {agent1.descripcion}")
plot(agent1)
print(f"Máxima proporcion: {agent1.list_stats[-1]}")
print("Valores Q para cada estado:\n", agent1.Q)
print("Política óptima obtenida\n", pi1, f"\n Acciones {actions1} \n Para el siguiente grid\n", env4.render())
print()

print(f"================== AGENTE 2 {agent2.descripcion}")
plot(agent2)
print(f"Máxima proporcion: {agent2.list_stats[-1]}")
print("Valores Q para cada estado:\n", agent2.Q)
print("Política óptima obtenida\n", pi2, f"\n Acciones {actions2} \n Para el siguiente grid\n", env4.render())
print()



AttributeError: 'FrozenAgentMC_On_All' object has no attribute 'descripcion'